In [8]:
from data_utils import LanesDataset, BDD100k
from torch.utils.data import DataLoader

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [139]:
class convBlock(nn.Module):
    """Some Information about convBlock"""
    def __init__(self, in_channels, out_channels, ks=3, stride=1, padding=1):
        super(convBlock, self).__init__()
        
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels, out_channels, ks, stride, padding)
        

    def forward(self, x):
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv(x)
        return x

In [143]:
class denseBlock2(nn.Module):
    """Some Information about denseBlock2"""
    def __init__(self, in_channels, out_channels, num_convs):
        super(denseBlock2, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.net = nn.Sequential()
        for i in range(num_convs):
            self.net.add_module(f'conv{i}', convBlock(self.in_channels, self.out_channels))
            self.in_channels = self.in_channels + self.out_channels

    def forward(self, x):
        for blk in self.net.children():
            # print(blk)
            y = blk(x)
            x = torch.cat([x, y], 1)
            print(x.shape)
        return y


In [6]:
train_img_dir = 'datasets/bdd100k/images/100k/train/'
train_label_dir = 'datasets/bdd100k/drivable_maps/labels/train/'
val_img_dir = 'datasets/bdd100k/images/100k/val/'
val_label_dir = 'datasets/bdd100k/drivable_maps/labels/val/'
model_dir = 'saved_models/'
exp_no = 2
logdir = 'runs/BDD100k_Experiment'+str(exp_no)
resize = (128,128)
SAMPLE_SIZE = 2000

In [9]:
# loading train data
train_dataset = BDD100k(train_img_dir,train_label_dir,resize=resize, transform=True, grayscale=False)
train_dataloader = DataLoader(train_dataset,batch_size=1, shuffle=False, pin_memory=True, num_workers=2)


In [16]:
img, mask = iter(train_dataloader).next()
print(img.shape)
print(mask.shape)

torch.Size([1, 2, 128, 128])

In [145]:
conv = nn.Conv2d(3,32,3,stride=1,padding=1)
# print(conv)
x = conv(img)
dense = denseBlock2(32, 32, 4)
y = dense(x)
print(y.shape)


torch.Size([1, 64, 128, 128])
torch.Size([1, 96, 128, 128])


RuntimeError: running_mean should contain 96 elements not 128